In [158]:
import boto3
import json
import time

# LocalStack endpoint (accessible from host)
ENDPOINT_URL = "http://localhost:4566"
REGION = "us-east-1"

# Resource Names
TASK_QUEUE_NAME = "task-queue"
NOTIFY_QUEUE_NAME = "notification-queue"
DLQ_QUEUE_NAME = "dlq-queue"
BUCKET_NAME = "results-bucket"

sqs = boto3.client("sqs", endpoint_url=ENDPOINT_URL, region_name=REGION)
s3 = boto3.client("s3", endpoint_url=ENDPOINT_URL, region_name=REGION)
lambdas = boto3.client("lambda", endpoint_url=ENDPOINT_URL, region_name=REGION)
iam = boto3.client("iam", endpoint_url=ENDPOINT_URL, region_name=REGION)
ssm = boto3.client("ssm", endpoint_url=ENDPOINT_URL, region_name=REGION)
dynamodb = boto3.client("dynamodb", endpoint_url=ENDPOINT_URL, region_name=REGION)


In [165]:
def create_or_get_queue(queue_name):
    try:
        return sqs.create_queue(QueueName=queue_name)['QueueUrl']
    except:
        return sqs.get_queue_url(QueueName=queue_name)['QueueUrl']

def get_queue_arn(queue_url):
    return sqs.get_queue_attributes(QueueUrl=queue_url, AttributeNames=['QueueArn'])['Attributes']['QueueArn']

def store_parameter(name, value):
    ssm.put_parameter(Name=name, Value=value, Type='String', Overwrite=True)

print("🏗️ Creating Infrastructure...")

try:
    s3.create_bucket(Bucket=BUCKET_NAME)
    print(f"✅ S3 Bucket: {BUCKET_NAME}")
except:
    print(f"✅ S3 Bucket: {BUCKET_NAME} (exists)")

TASK_QUEUE_URL = create_or_get_queue(TASK_QUEUE_NAME)
NOTIFY_QUEUE_URL = create_or_get_queue(NOTIFY_QUEUE_NAME)
DLQ_URL = create_or_get_queue(DLQ_QUEUE_NAME)
print(f"✅ Queues created")

try:
    ROLE_ARN = iam.create_role(
        RoleName="lambda-role",
        AssumeRolePolicyDocument=json.dumps({
            "Version": "2012-10-17",
            "Statement": [{
                "Effect": "Allow",
                "Principal": {"Service": "lambda.amazonaws.com"},
                "Action": "sts:AssumeRole"
            }]
        })
    )['Role']['Arn']
except:
    ROLE_ARN = iam.get_role(RoleName="lambda-role")['Role']['Arn']
print(f"✅ IAM Role created")

sqs.set_queue_attributes(
    QueueUrl=TASK_QUEUE_URL,
    Attributes={
        'RedrivePolicy': json.dumps({
            'deadLetterTargetArn': get_queue_arn(DLQ_URL),
            'maxReceiveCount': '2'
        })
    }
)
print(f"✅ DLQ configured")

# Create DynamoDB table
try:
    dynamodb.create_table(
        TableName='orders',
        KeySchema=[{'AttributeName': 'order_id', 'KeyType': 'HASH'}],
        AttributeDefinitions=[{'AttributeName': 'order_id', 'AttributeType': 'S'}],
        BillingMode='PAY_PER_REQUEST'
    )
    print(f"✅ DynamoDB table created")
except:
    print(f"✅ DynamoDB table exists")

# Store in Parameter Store with different names
store_parameter("poc-lambda-role-arn", ROLE_ARN)
store_parameter("poc-task-queue-url", TASK_QUEUE_URL)
store_parameter("poc-notification-queue-url", NOTIFY_QUEUE_URL)
store_parameter("poc-dlq-queue-url", DLQ_URL)
store_parameter("poc-results-bucket-name", BUCKET_NAME)
store_parameter("poc-orders-table-name", "orders")
print(f"✅ Parameters stored\n")


🏗️ Creating Infrastructure...
✅ S3 Bucket: results-bucket
✅ Queues created
✅ IAM Role created
✅ DLQ configured
✅ DynamoDB table created
✅ Parameters stored



In [167]:
import zipfile
import io
import os

def get_parameter(name):
    return ssm.get_parameter(Name=name)['Parameter']['Value']

def create_lambda_zip():
    zip_buffer = io.BytesIO()
    with zipfile.ZipFile(zip_buffer, "a", zipfile.ZIP_DEFLATED, False) as zip_file:
        print("📦 Packaging files for Lambda...\n")
        
        for root, dirs, files in os.walk('../app'):
            for file in files:
                if file.endswith('.py'):
                    file_path = os.path.join(root, file)
                    arcname = os.path.relpath(file_path, start='..')
                    zip_file.write(file_path, arcname)
                    print(f"  ✅ {arcname}")
        
        for root, dirs, files in os.walk('../lambdas'):
            for file in files:
                if file.endswith('.py'):
                    file_path = os.path.join(root, file)
                    zip_file.write(file_path, arcname=file)
                    print(f"  ✅ {file}")
        
        print(f"\n📊 Total size: {zip_buffer.tell()} bytes\n")
    return zip_buffer.getvalue()

zip_content = create_lambda_zip()

# Get Role ARN from Parameter Store 
ROLE_ARN = get_parameter("poc-lambda-role-arn")

for func in ["task_lambda", "notification_lambda", "dlq_processor_lambda"]:
    try: lambdas.delete_function(FunctionName=func)
    except: pass

LAMBDA_ENV = {
    'Variables': {
        'AWS_ENDPOINT_URL': 'http://localstack:4566',
        'AWS_REGION': REGION
    }
}

lambdas.create_function(
    FunctionName="task_lambda",
    Runtime="python3.10",
    Role=ROLE_ARN,
    Handler="task_lambda.lambda_handler",
    Code={'ZipFile': zip_content},
    Environment=LAMBDA_ENV,
    Timeout=30
)

lambdas.create_function(
    FunctionName="notification_lambda",
    Runtime="python3.10",
    Role=ROLE_ARN,
    Handler="notification_lambda.lambda_handler",
    Code={'ZipFile': zip_content},
    Environment=LAMBDA_ENV,
    Timeout=30
)

lambdas.create_function(
    FunctionName="dlq_processor_lambda",
    Runtime="python3.10",
    Role=ROLE_ARN,
    Handler="dlq_processor_lambda.lambda_handler",
    Code={'ZipFile': zip_content},
    Environment=LAMBDA_ENV,
    Timeout=30
)

print("✅ Lambdas deployed!\n")


📦 Packaging files for Lambda...

  ✅ app\auth.py
  ✅ app\config.py
  ✅ app\database.py
  ✅ app\notifier.py
  ✅ app\parameter_store.py
  ✅ app\processors.py
  ✅ app\storage.py
  ✅ app\__init__.py
  ✅ app\helpers\discount_calculator.py
  ✅ app\helpers\__init__.py
  ✅ api_lambda.py
  ✅ authorizer_lambda.py
  ✅ dlq_processor_lambda.py
  ✅ notification_lambda.py
  ✅ task_lambda.py

📊 Total size: 8760 bytes

✅ Lambdas deployed!



In [168]:
def add_trigger(queue_url, function_name):
    try:
        queue_arn = sqs.get_queue_attributes(QueueUrl=queue_url, AttributeNames=['QueueArn'])['Attributes']['QueueArn']
        lambdas.create_event_source_mapping(
            EventSourceArn=queue_arn,
            FunctionName=function_name,
            BatchSize=10
        )
        print(f"🔗 Linked: {function_name}")
    except:
        print(f"🔗 Already linked: {function_name}")

# Get URLs from Parameter Store - FIXED NAMES
TASK_QUEUE_URL = get_parameter("poc-task-queue-url")
NOTIFY_QUEUE_URL = get_parameter("poc-notification-queue-url")
DLQ_URL = get_parameter("poc-dlq-queue-url")

add_trigger(TASK_QUEUE_URL, "task_lambda")
add_trigger(NOTIFY_QUEUE_URL, "notification_lambda")
add_trigger(DLQ_URL, "dlq_processor_lambda")


🔗 Linked: task_lambda
🔗 Linked: notification_lambda
🔗 Linked: dlq_processor_lambda


In [169]:
import time
import json

# Order Input
order_id = f"ORD-{int(time.time())}"

order_message = {
    "order_id": order_id,
    "items": [
        {"name": "Laptop", "price": 200.99, "quantity": 1},
        {"name": "Mouse", "price": 30.99, "quantity": 2},
        {"name": "Keyboard", "price": 50.00, "quantity": 1}

    ],
    "promo_code": "SAVE10"
}

sqs.send_message(
    QueueUrl=TASK_QUEUE_URL,
    MessageBody=json.dumps(order_message)
)

print(f"📤 Order sent: {order_id}")
print(f"   Items: {len(order_message['items'])} items")
print(f"   Promo: {order_message['promo_code']}")
print("⏳ Waiting for processing (10s)...")
time.sleep(10)

# Check S3 for invoice
try:
    obj = s3.get_object(Bucket=BUCKET_NAME, Key=f"{order_id}.json")
    invoice = json.loads(obj['Body'].read())
    print(f"\n INVOICE FOUND in S3:")
    print(f"  Order ID: {invoice['order_id']}")
    print(f"  Subtotal: ${invoice['subtotal']}")
    print(f"  Discount: ${invoice['discount']}")
    print(f"  Final Total: ${invoice['final_total']}")
    print(f"  Status: {invoice['status']}")
except Exception as e:
    print(f" Invoice not found: {e}")



📤 Order sent: ORD-1771233043
   Items: 3 items
   Promo: SAVE10
⏳ Waiting for processing (10s)...

 INVOICE FOUND in S3:
  Order ID: ORD-1771233043
  Subtotal: $312.97
  Discount: $31.3
  Final Total: $281.67
  Status: completed


In [146]:
import time
import json

# Order Input
order_id = f"ORD-{int(time.time())}"

order_message = {
    "order_id": order_id,
    "correlation_id": f"corr-{int(time.time())}",
    "items": [
        {"name": "Laptop", "price": 200.99, "quantity": 1},
        {"name": "Mouse", "price": 30.99, "quantity": 2},
        {"name": "Keyboard", "price": 50.00, "quantity": 1}
    ],
    "promo_code": "SAVE10"
}

sqs.send_message(
    QueueUrl=TASK_QUEUE_URL,
    MessageBody=json.dumps(order_message)
)

print(f"📤 Order sent: {order_id}")
print(f"   Items: {len(order_message['items'])} items")
print(f"   Promo: {order_message['promo_code']}")
print("⏳ Waiting for processing (10s)...")
time.sleep(10)

# Check S3 for invoice
try:
    obj = s3.get_object(Bucket=BUCKET_NAME, Key=f"{order_id}.json")
    invoice = json.loads(obj['Body'].read())
    print(f"\n✅ INVOICE FOUND in S3:")
    print(f"  Order ID: {invoice['order_id']}")
    print(f"  Correlation ID: {invoice.get('correlation_id', 'N/A')}")
    print(f"  Subtotal: ${invoice['subtotal']}")
    print(f"  Discount: ${invoice['discount']}")
    print(f"  Final Total: ${invoice['final_total']}")
    print(f"  Bulk Discount: ${invoice.get('bulk_discount', 0):.2f}")
    print(f"  Tax: ${invoice.get('tax', 0):.2f}")
    print(f"  Status: {invoice['status']}")
    
    # ✅ VERIFY NESTED MODULE ACCESS
    if 'bulk_discount' in invoice and 'tax' in invoice:
        print(f"\n✅ NESTED MODULE VERIFIED:")
        print(f"   📂 app/helpers/discount_calculator.py was accessed!")
        print(f"   🔹 calculate_bulk_discount() returned: ${invoice['bulk_discount']:.2f}")
        print(f"   🔹 calculate_tax() returned: ${invoice['tax']:.2f}")
    else:
        print(f"\n❌ NESTED MODULE NOT ACCESSED")
    
    print(f"\n🎉 Order processed successfully!")
except Exception as e:
    print(f"❌ Invoice not found: {e}")

# Check LocalStack logs for detailed module access
print(f"\n💡 To see detailed logs, run:")
print(f"   docker logs localstack_main | grep '{order_id}'")


📤 Order sent: ORD-1770975337
   Items: 3 items
   Promo: SAVE10
⏳ Waiting for processing (10s)...

✅ INVOICE FOUND in S3:
  Order ID: ORD-1770975337
  Correlation ID: corr-1770975337
  Subtotal: $312.97
  Discount: $31.3
  Final Total: $281.67
  Bulk Discount: $15.65
  Tax: $22.53
  Status: completed

✅ NESTED MODULE VERIFIED:
   📂 app/helpers/discount_calculator.py was accessed!
   🔹 calculate_bulk_discount() returned: $15.65
   🔹 calculate_tax() returned: $22.53

🎉 Order processed successfully!

💡 To see detailed logs, run:
   docker logs localstack_main | grep 'ORD-1770975337'


In [38]:
# logs (application logs), request -> use models (validate it ) , Authenication JWT  (any other inputs also fine) add the correlaeation Id ()  try to send multiple msgs to the queue ( inputs)

In [ ]:
#DL queue -> DL queue , identify  the sqs message -> how does the message will trigger the lambda function ->, sqs trigger lambda queue please check it , reseacch about the DLQs ,  

In [ ]:
#  make some modules and check it ,  


✅ Sent 10 orders - Lambda will process in batch


In [157]:
import time
import json
import boto3

ENDPOINT_URL = "http://localhost:4566"
REGION = "us-east-1"

sqs = boto3.client("sqs", endpoint_url=ENDPOINT_URL, region_name=REGION)
s3 = boto3.client("s3", endpoint_url=ENDPOINT_URL, region_name=REGION)
ssm = boto3.client("ssm", endpoint_url=ENDPOINT_URL, region_name=REGION)

# Get values from Parameter Store
TASK_QUEUE_URL = ssm.get_parameter(Name="poc-task-queue-url")['Parameter']['Value']
DLQ_URL = ssm.get_parameter(Name="poc-dlq-queue-url")['Parameter']['Value']
BUCKET_NAME = ssm.get_parameter(Name="poc-results-bucket-name")['Parameter']['Value']

# Bulk Order Input - 10 orders with variety + 1 negative test case
bulk_orders = [
    {
        "items": [
            {"name": "Laptop", "price": 999.99, "quantity": 1},
            {"name": "Mouse", "price": 29.99, "quantity": 1}
        ],
        "promo_code": "SAVE10"
    },
    {
        "items": [
            {"name": "Monitor", "price": 299.99, "quantity": 2}
        ],
        "promo_code": "SAVE20"
    },
    {
        "items": [
            {"name": "Keyboard", "price": 79.99, "quantity": 1},
            {"name": "Headphones", "price": 149.99, "quantity": 1}
        ],
        "promo_code": None
    },
    {
        "items": [
            {"name": "Webcam", "price": 89.99, "quantity": 1}
        ],
        "promo_code": "FREESHIP"
    },
    {
        "items": [
            {"name": "Desk Chair", "price": 249.99, "quantity": 1},
            {"name": "Desk Lamp", "price": 39.99, "quantity": 1}
        ],
        "promo_code": "SAVE30"
    },
    {
        "items": [
            {"name": "USB Hub", "price": 39.99, "quantity": 2},
            {"name": "Cable", "price": 15.99, "quantity": 3}
        ],
        "promo_code": "SAVE10"
    },
    {
        "items": [
            {"name": "Speaker", "price": 79.99, "quantity": 1}
        ],
        "promo_code": "SAVE20"
    },
    {
        "items": [
            {"name": "Tablet", "price": 399.99, "quantity": 1},
            {"name": "Case", "price": 29.99, "quantity": 1}
        ],
        "promo_code": "SAVE10"
    },
    {
        "items": [
            {"name": "Phone", "price": 699.99, "quantity": 1}
        ],
        "promo_code": "SAVE20"
    },
    {
        "items": [
            {"name": "Charger", "price": 49.99, "quantity": 2},
            {"name": "Screen Protector", "price": 19.99, "quantity": 1}
        ],
        "promo_code": "FREESHIP"
    },
    # NEGATIVE TEST CASE - This will trigger DLQ
    {
        "items": [
            {"name": "BadItem", "price": -100.00, "quantity": 1}  # Negative price triggers Lambda failure
        ],
        "promo_code": "SAVE10"
    }
]

order_ids = []
print(f"📤 Sending {len(bulk_orders)} bulk orders (10 good + 1 bad)...")

for i, order_data in enumerate(bulk_orders, 1):
    order_id = f"BULK-{int(time.time())}-{i:02d}"
    order_ids.append(order_id)
    
    order_message = {
        "order_id": order_id,
        "items": order_data["items"],
        "promo_code": order_data.get("promo_code")
    }
    
    sqs.send_message(QueueUrl=TASK_QUEUE_URL, MessageBody=json.dumps(order_message))
    
    items_count = len(order_data["items"])
    promo = order_data.get("promo_code", "None")
    is_bad = order_data["items"][0]["price"] < 0
    status = "❌ BAD" if is_bad else "✅"
    print(f"{status} Sent: {order_id} - {items_count} items, Promo: {promo}")
    time.sleep(0.1)

print(f"\n📤 Total sent: {len(bulk_orders)} orders")

# Wait for initial processing
print("⏳ Waiting 15s for initial processing...")
time.sleep(15)

# Check DLQ status
print("\n🔍 Checking DLQ status...")
try:
    attrs = sqs.get_queue_attributes(
        QueueUrl=DLQ_URL,
        AttributeNames=['ApproximateNumberOfMessages']
    )
    dlq_count = int(attrs['Attributes'].get('ApproximateNumberOfMessages', 0))
    print(f"   Messages in DLQ: {dlq_count}")
    
    if dlq_count > 0:
        print(f"   ⚠️ Found {dlq_count} message(s) in DLQ, waiting 20s more for DLQ processing...")
        time.sleep(20)
    else:
        print("   ℹ️ No messages in DLQ yet, waiting 10s more...")
        time.sleep(10)
except Exception as e:
    print(f"   ❌ Error checking DLQ: {e}")
    time.sleep(10)

# Check S3 for all invoices
print("\n" + "="*60)
print("📊 BULK PROCESSING RESULTS:")
print("="*60)

success_count = 0
failed_count = 0
dlq_recovered = 0
total_revenue = 0

for order_id in order_ids:
    try:
        obj = s3.get_object(Bucket=BUCKET_NAME, Key=f"{order_id}.json")
        invoice = json.loads(obj['Body'].read())
        
        total_revenue += invoice['final_total']
        
        if invoice.get('recovered_from_dlq'):
            dlq_recovered += 1
            print(f"🔧 {order_id}: ${invoice['final_total']:.2f} (RECOVERED FROM DLQ)")
        else:
            success_count += 1
            print(f"✅ {order_id}: ${invoice['final_total']:.2f} (Items: {len(invoice['items'])}, Discount: ${invoice['discount']:.2f})")
        
    except Exception as e:
        failed_count += 1
        print(f"❌ {order_id}: Not found")

print("="*60)
print(f"📈 Summary:")
print(f"   Total Orders: {len(bulk_orders)}")
print(f"   Successful: {success_count}")
print(f"   Recovered from DLQ: {dlq_recovered}")
print(f"   Failed: {failed_count}")
print(f"   Success Rate: {((success_count + dlq_recovered)/len(bulk_orders)*100):.1f}%")
print(f"   Total Revenue: ${total_revenue:.2f}")

# Final DLQ check
try:
    attrs = sqs.get_queue_attributes(
        QueueUrl=DLQ_URL,
        AttributeNames=['ApproximateNumberOfMessages']
    )
    dlq_count = int(attrs['Attributes'].get('ApproximateNumberOfMessages', 0))
    print(f"   Messages remaining in DLQ: {dlq_count}")
except:
    print("   DLQ check failed")

print("="*60)


📤 Sending 11 bulk orders (10 good + 1 bad)...
✅ Sent: BULK-1771231149-01 - 2 items, Promo: SAVE10
✅ Sent: BULK-1771231149-02 - 1 items, Promo: SAVE20
✅ Sent: BULK-1771231149-03 - 2 items, Promo: None
✅ Sent: BULK-1771231150-04 - 1 items, Promo: FREESHIP
✅ Sent: BULK-1771231150-05 - 2 items, Promo: SAVE30
✅ Sent: BULK-1771231150-06 - 2 items, Promo: SAVE10
✅ Sent: BULK-1771231150-07 - 1 items, Promo: SAVE20
✅ Sent: BULK-1771231150-08 - 2 items, Promo: SAVE10
✅ Sent: BULK-1771231150-09 - 1 items, Promo: SAVE20
✅ Sent: BULK-1771231151-10 - 2 items, Promo: FREESHIP
❌ BAD Sent: BULK-1771231151-11 - 1 items, Promo: SAVE10

📤 Total sent: 11 orders
⏳ Waiting 15s for initial processing...

🔍 Checking DLQ status...
   Messages in DLQ: 0
   ℹ️ No messages in DLQ yet, waiting 10s more...

📊 BULK PROCESSING RESULTS:
✅ BULK-1771231149-01: $926.98 (Items: 2, Discount: $103.00)
✅ BULK-1771231149-02: $479.98 (Items: 1, Discount: $120.00)
✅ BULK-1771231149-03: $229.98 (Items: 2, Discount: $0.00)
✅ BULK-

In [ ]:
# name the handler function (test it ) ->if any other names and check it once   , authentication : request -> BFF(another microservice) -> token -> (message ) -> Lambda function

In [143]:
import boto3
import json

ENDPOINT_URL = "http://localhost:4566"
REGION = "us-east-1"

sqs = boto3.client("sqs", endpoint_url=ENDPOINT_URL, region_name=REGION)
ssm = boto3.client("ssm", endpoint_url=ENDPOINT_URL, region_name=REGION)
lambdas = boto3.client("lambda", endpoint_url=ENDPOINT_URL, region_name=REGION)

# Get DLQ URL from Parameter Store
DLQ_URL = ssm.get_parameter(Name="poc-dlq-queue-url")['Parameter']['Value']

print("=" * 60)
print("🔍 DLQ DIAGNOSTICS")
print("=" * 60)

# Check DLQ messages
try:
    attrs = sqs.get_queue_attributes(
        QueueUrl=DLQ_URL,
        AttributeNames=['ApproximateNumberOfMessages', 'ApproximateNumberOfMessagesNotVisible']
    )
    msg_count = int(attrs['Attributes'].get('ApproximateNumberOfMessages', 0))
    in_flight = int(attrs['Attributes'].get('ApproximateNumberOfMessagesNotVisible', 0))
    
    print(f"\n📊 DLQ Stats:")
    print(f"   Messages available: {msg_count}")
    print(f"   Messages in-flight: {in_flight}")
    
    if msg_count > 0:
        print(f"\n📬 Fetching messages from DLQ...")
        response = sqs.receive_message(
            QueueUrl=DLQ_URL,
            MaxNumberOfMessages=10,
            VisibilityTimeout=30,
            WaitTimeSeconds=1
        )
        
        messages = response.get('Messages', [])
        print(f"   Found {len(messages)} messages")
        
        for i, msg in enumerate(messages, 1):
            body = json.loads(msg['Body'])
            print(f"\n   Message {i}:")
            print(f"      Order ID: {body.get('order_id')}")
            print(f"      Items: {body.get('items')}")
            
except Exception as e:
    print(f"❌ Error checking DLQ: {e}")

# Check Lambda event source mapping
print(f"\n🔗 Lambda Event Source Mappings:")
try:
    mappings = lambdas.list_event_source_mappings(FunctionName="dlq_processor_lambda")
    for mapping in mappings.get('EventSourceMappings', []):
        print(f"   UUID: {mapping['UUID']}")
        print(f"   State: {mapping['State']}")
        print(f"   Enabled: {mapping.get('State') == 'Enabled'}")
        print(f"   Last Processing Result: {mapping.get('LastProcessingResult', 'N/A')}")
except Exception as e:
    print(f"   ❌ Error: {e}")

print("\n" + "=" * 60)


🔍 DLQ DIAGNOSTICS

📊 DLQ Stats:
   Messages available: 0
   Messages in-flight: 0

🔗 Lambda Event Source Mappings:
   UUID: 12dcd95c-9d78-499d-8e3d-fb9d1453ffc1
   State: Enabled
   Enabled: True
   Last Processing Result: N/A



🧪 Testing DLQ with UNFIXABLE order...
📤 Sending unfixable order: UNFIXABLE-1770972637
   Issue: Empty items array (unfixable)
⏳ Waiting 30s for retries + DLQ...
❌ DLQ check failed: An error occurred (AWS.SimpleQueueService.NonExistentQueue) when calling the GetQueueAttributes operation: The specified queue does not exist.
❌ UNEXPECTED: Order was processed despite being unfixable!
